# ANN神经网络权重部署

In [1]:
import numpy as np

3bit（8级别）差分矩阵元素对应表

8级别的衰减值分别为：-1dB，-1.66dB，-2.44dB，-3.39dB，-4.61dB，-6.30dB，-9.14dB，-20dB

为了确保传输率的增量是线性的，dB值就不能是线性的

| 权重 | 差分正极dB    | 对应传输率 | 差分负极dB     | 对应传输率  |
|------|--------------|-----------|---------------|-------------|
|7     | -1.00dB      |   0.794   |    -20.0dB     |    0.010   |
|6     | -1.00dB      |   0.794   |    -9.14dB     |    0.122   |
|5     | -1.00dB      |   0.794   |    -6.30dB     |    0.234   |
|4     | -1.00dB      |   0.794   |    -4.61dB     |    0.346   |
|3     | -1.00dB      |   0.794   |    -3.39dB     |    0.458   |
|2     | -1.00dB      |   0.794   |    -2.44dB     |    0.570   |
|1     | -1.00dB      |   0.794   |    -1.66dB     |    0.682   |
|0     | -1.00dB      |   0.794   |    -1dB        |    0.794   |
|-1    | -1.66dB      |   0.682   |    -1dB        |    0.794   |
|-2    | -2.44dB      |   0.570   |    -1dB        |    0.794   |
|-3    | -3.39dB      |   0.458   |    -1dB        |    0.794   |
|-4    | -4.61dB      |   0.346   |    -1dB        |    0.794   |
|-5    | -6.30dB      |   0.234   |    -1dB        |    0.794   |
|-6    | -9.14dB      |   0.122   |    -1dB        |    0.794   |
|-7    | -20.0dB      |   0.010   |    -1dB        |    0.794   |

In [2]:
# 正极权重映射dB对应表
mapping_dict_pos = {7: -1, 6: -1, 5: -1, 4: -1, 3:-1, 2:-1, 1:-1, 0:-1, 
                    -1:-1.66, -2:-2.44, -3:-3.39, -4:-4.61, -5:-6.30, -6:-9.14, -7:-20}
# 负极权重映射dB对应表
mapping_dict_neg = {7: -20, 6: -9.14, 5: -6.30, 4: -4.61, 3:-3.39, 2:-2.44, 1:-1.66, 0:-1,
                    -1:-1, -2:-1, -3:-1, -4:-1, -5:-1, -6:-1, -7:-1}

def weight_to_dB(weight, mapping_dict_pos, mapping_dict_neg):
    """
    将输入的numpy数组按照给定的映射表转换为新的数组。

    参数:
        weight (numpy.array): 输入的numpy数组。
        mapping_dict (dict): 映射表，键是输入数组中的值，值是映射后的值。

    返回:
        W_p_dB: 映射后的正极dB数组。
        W_n_dB: 映射后的负极dB数组。
    """
    # 创建2个与输入数组形状相同的空数组，用于存储映射后的值
    W_p_dB = np.zeros_like(weight)
    W_n_dB = np.zeros_like(weight)

    # 遍历输入数组的每个元素，并根据正极映射表更新输出数组
    for key, value in mapping_dict_pos.items():
        W_p_dB[weight == key] = value

    # 遍历输入数组的每个元素，并根据负极映射表更新输出数组
    for key, value in mapping_dict_neg.items():
        W_n_dB[weight == key] = value

    return W_p_dB, W_n_dB

## 神经网络第1层量化后权重部署

In [3]:
print("-----神经网络第1层量化后权重部署-----")

# 测试输入向量
x = np.array([1.0000, 0.7000, 0.2600, 0.0600])
print("输入向量功率 = \n", x)

# 原始权重矩阵数据
W = np.array([[ 1., -4.,  6.,  5.],
              [ 5.,  4., -5., -7.],
              [ 7.,  5., -5., -6.],
              [ 1., -4.,  5.,  4.]])
print("原始权重矩阵数据 = \n", W)

# 将pytorch权重矩阵转置
W = W.T
print("将pytorch权重矩阵转置 = \n", W)

# 原始矩阵乘法输出值
y = np.matmul(x, W)
print("原始矩阵乘法输出结果 = ", y)

# 将权重映射到dB
W_p_dB, W_n_dB = weight_to_dB(W, mapping_dict_pos, mapping_dict_neg)

# 差分正极dB矩阵
# W_p_dB = np.array([[-1, -1, -3.39, -20],
#                    [-1, -1, -3.39, -3.39],
#                    [-1, -1, -1, -3.39],
#                    [-1, -1, -1, -1]])
print("差分正极dB矩阵 = \n", W_p_dB)

# 差分负极dB矩阵
# W_n_dB = np.array([[-2.44, -3.39, -1, -1],
#                    [-2.44, -3.39, -1, -1],
#                    [-1.66, -1, -2.44, -1],
#                    [-2.44, -1, -1.66, -2.44]])
print("差分负极dB矩阵 = \n", W_n_dB)

# 差分正极传输率矩阵
# W_p = np.array([[0.794, 0.794, 0.199, 0.01],
#                 [0.794, 0.794, 0.199, 0.199],
#                 [0.794, 0.794, 0.794, 0.199],
#                 [0.794, 0.794, 0.794, 0.794]])

# 差分负极传输率矩阵
# W_n = np.array([[0.316, 0.199, 0.794, 0.794],
#                 [0.316, 0.199, 0.794, 0.794],
#                 [0.501, 0.794, 0.316, 0.794],
#                 [0.316, 0.794, 0.501, 0.316]])

# 传输率转化为dB
# W_p_dB = 10 * np.log10(W_p / 1)
# W_n_dB = 10 * np.log10(W_n / 1)
# print("差分正极矩阵传输率转化为dB = \n", W_p_dB)
# print("差分负极矩阵传输率转化为dB = \n", W_n_dB)

# dB转化为传输率
W_p = np.power(10, W_p_dB / 10)
W_n = np.power(10, W_n_dB / 10)
print("差分正极矩阵dB转化为传输率 = \n", W_p)
print("差分负极矩阵dB转化为传输率 = \n", W_n)

# 差分矩阵乘法
y_p = np.matmul(x, W_p)
y_n = np.matmul(x, W_n)

# 差分输出结果
print("差分正极输出结果x * W_p =", y_p)
print("差分负极输出结果x * W_n =", y_n)

# 差分矩阵结果相减后输出
y_diff = y_p - y_n
print("差分矩阵结果相减后输出 = ", y_diff)

# 考虑差分交叉阵列结构平均分配功率
y_out_p = y_p / 32
y_out_n = y_n / 32
print("考虑差分交叉阵列结构除以32后的实际差分正极输出光功率值y_out_p =\n", y_out_p)
print("考虑差分交叉阵列结构除以32后的实际差分负极输出光功率值y_out_n =\n", y_out_n)

# 实际差分输出功率相减后输出值
y_out_diff = y_out_p - y_out_n
print("考虑差分交叉阵列结构除以32后的实际差分输出功率相减后输出值y_out_diff =\n", y_out_diff)

# 考虑前端放大10K倍，将光功率转化为电压信号，输入到电路中进行仿真
vin = y_out_diff * 10e3
print(f"考虑前端放大10K倍，将光功率转化为电压信号=\n{vin}")

# 输出功率转化为dBm
y_dBm_p = 10 * np.log10(y_out_p / 1e-3)
y_dBm_n = 10 * np.log10(y_out_n / 1e-3)
print("差分正极转化为dBm的值 = ", y_dBm_p)
print("差分负极转化为dBm的值 = ", y_dBm_n)

# 差分输出功率dBm相减后输出值
y_diff_dBm = y_dBm_p - y_dBm_n
print("差分输出功率dBm相减后输出值 = ", y_diff_dBm)

-----神经网络第1层量化后权重部署-----
输入向量功率 = 
 [1.   0.7  0.26 0.06]
原始权重矩阵数据 = 
 [[ 1. -4.  6.  5.]
 [ 5.  4. -5. -7.]
 [ 7.  5. -5. -6.]
 [ 1. -4.  5.  4.]]
将pytorch权重矩阵转置 = 
 [[ 1.  5.  7.  1.]
 [-4.  4.  5. -4.]
 [ 6. -5. -5.  5.]
 [ 5. -7. -6.  4.]]
原始矩阵乘法输出结果 =  [ 0.06  6.08  8.84 -0.26]
差分正极dB矩阵 = 
 [[ -1.    -1.    -1.    -1.  ]
 [ -4.61  -1.    -1.    -4.61]
 [ -1.    -6.3   -6.3   -1.  ]
 [ -1.   -20.    -9.14  -1.  ]]
差分负极dB矩阵 = 
 [[ -1.66  -6.3  -20.    -1.66]
 [ -1.    -4.61  -6.3   -1.  ]
 [ -9.14  -1.    -1.    -6.3 ]
 [ -6.3   -1.    -1.    -4.61]]
差分正极矩阵dB转化为传输率 = 
 [[0.79432823 0.79432823 0.79432823 0.79432823]
 [0.34593938 0.79432823 0.79432823 0.34593938]
 [0.79432823 0.23442288 0.23442288 0.79432823]
 [0.79432823 0.01       0.12189896 0.79432823]]
差分负极矩阵dB转化为传输率 = 
 [[0.68233869 0.23442288 0.01       0.68233869]
 [0.79432823 0.34593938 0.23442288 0.79432823]
 [0.12189896 0.79432823 0.79432823 0.23442288]
 [0.23442288 0.79432823 0.79432823 0.34593938]]
差分正极输出结果x * W_p = [1.290

## 神经网络第2层量化后权重部署

In [4]:
print("-----神经网络第2层量化后权重部署-----")

# 测试输入向量
x = np.array([0.00020448, 0.0212857, 0.03094815, 0])
print("输入向量功率 = \n", x)

# 原始权重矩阵数据
W = np.array([[-4.,  5.,  5., -5.],
                [ 1.,  4.,  3.,  0.],
                [ 6., -7., -6.,  6.]])
print("原始权重矩阵数据 = \n", W)

# 将pytorch权重矩阵转置
W = W.T
print("将pytorch权重矩阵转置 = \n", W)

# 将权重映射到dB
W_p_dB, W_n_dB = weight_to_dB(W, mapping_dict_pos, mapping_dict_neg)

# 差分正极dB矩阵
print("差分正极dB矩阵 = \n", W_p_dB)

# 差分负极dB矩阵
print("差分负极dB矩阵 = \n", W_n_dB)

# dB转化为传输率
W_p = np.power(10, W_p_dB / 10)
W_n = np.power(10, W_n_dB / 10)
print("差分正极矩阵dB转化为传输率 = \n", W_p)
print("差分负极矩阵dB转化为传输率 = \n", W_n)

# 差分矩阵乘法
y_p = np.matmul(x, W_p)
y_n = np.matmul(x, W_n)

# 差分输出结果
print("差分正极输出结果x * W_p =", y_p)
print("差分负极输出结果x * W_n =", y_n)

# 差分矩阵结果相减后输出
y_diff = y_p - y_n
print("差分矩阵结果相减后输出 = ", y_diff)

# 考虑差分交叉阵列结构平均分配功率
y_out_p = y_p / 32
y_out_n = y_n / 32
print("考虑差分交叉阵列结构除以32后的实际差分正极输出光功率值y_out_p =\n", y_out_p)
print("考虑差分交叉阵列结构除以32后的实际差分负极输出光功率值y_out_n =\n", y_out_n)

# 实际差分输出功率相减后输出值
y_out_diff = y_out_p - y_out_n
print("考虑差分交叉阵列结构除以32后的实际差分输出功率相减后输出值y_out_diff =\n", y_out_diff)

# 考虑前端放大10K倍，将光功率转化为电压信号，输入到电路中进行仿真
vin = y_out_diff * 10e3
print(f"考虑前端放大10K倍，将光功率转化为电压信号=\n{vin}")

# 输出功率转化为dBm
y_dBm_p = 10 * np.log10(y_out_p / 1e-3)
y_dBm_n = 10 * np.log10(y_out_n / 1e-3)
print("差分正极转化为dBm的值 = ", y_dBm_p)
print("差分负极转化为dBm的值 = ", y_dBm_n)

# 差分输出功率dBm相减后输出值
y_diff_dBm = y_dBm_p - y_dBm_n
print("差分输出功率dBm相减后输出值 = ", y_diff_dBm)

-----神经网络第2层量化后权重部署-----
输入向量功率 = 
 [0.00020448 0.0212857  0.03094815 0.        ]
原始权重矩阵数据 = 
 [[-4.  5.  5. -5.]
 [ 1.  4.  3.  0.]
 [ 6. -7. -6.  6.]]
将pytorch权重矩阵转置 = 
 [[-4.  1.  6.]
 [ 5.  4. -7.]
 [ 5.  3. -6.]
 [-5.  0.  6.]]
差分正极dB矩阵 = 
 [[ -4.61  -1.    -1.  ]
 [ -1.    -1.   -20.  ]
 [ -1.    -1.    -9.14]
 [ -6.3   -1.    -1.  ]]
差分负极dB矩阵 = 
 [[-1.   -1.66 -9.14]
 [-6.3  -4.61 -1.  ]
 [-6.3  -3.39 -1.  ]
 [-1.   -1.   -9.14]]
差分正极矩阵dB转化为传输率 = 
 [[0.34593938 0.79432823 0.79432823]
 [0.79432823 0.79432823 0.01      ]
 [0.79432823 0.79432823 0.12189896]
 [0.23442288 0.79432823 0.79432823]]
差分负极矩阵dB转化为传输率 = 
 [[0.79432823 0.68233869 0.12189896]
 [0.23442288 0.34593938 0.79432823]
 [0.23442288 0.45814189 0.79432823]
 [0.79432823 0.79432823 0.12189896]]
差分正极输出结果x * W_p = [0.04156156 0.04165325 0.00414783]
差分负极输出结果x * W_n = [0.01240723 0.02168173 0.04151575]
差分矩阵结果相减后输出 =  [ 0.02915433  0.01997152 -0.03736792]
考虑差分交叉阵列结构除以32后的实际差分正极输出光功率值y_out_p =
 [0.0012988  0.00130166 0.00012962